# Scripts to manipulate dataset

In [0]:
import shutil
import os

In [0]:
def split_test_into_subfolders(path):
    
    dest = 'sc5-Test-tensorflow'
    if not os.path.exists(dest):
        os.makedirs(dest)
    
    with open(path) as file:
        
        for line in file:
            file_name, class_name = line.split(';')
            class_name = class_name.replace('\n','')
            
            subdest = dest + '/' + class_name
            if not os.path.exists(subdest):
                os.makedirs(subdest)
            shutil.copy('sc5-Test/'+file_name, subdest)
            
        file.close()

In [0]:
def split_test_into_family_class(path):
    
    dest = 'sc5-test-tensorflow'
    start = 'sc5-Test-tensorflow-tmp'
    if not os.path.exists(dest):
        os.makedirs(dest)
        
    with open(path) as file:
        
        for line in file:
            file_name, class_name = line.split(';')
            class_name = class_name.replace('\n','')
            
            if os.path.exists(start+'/'+file_name):
                subdest = dest + '/' + class_name
                
                if not os.path.exists(subdest):
                    os.makedirs(subdest)
                shutil.move(start+'/'+file_name,subdest)
        file.close()  

In [0]:
def split_into_family_class(path):
    
    dest = 'sc5-tensorflow'
    if not os.path.exists(dest):
        os.makedirs(dest)
        
    with open(path) as file:
        
        for line in file:
            file_name, class_name = line.split(';')
            class_name = class_name.replace('\n','')
            
            if os.path.exists('sc5/'+file_name):
                subdest = dest + '/' + class_name
                
                if not os.path.exists(subdest):
                    os.makedirs(subdest)
                shutil.move('sc5/'+file_name,subdest)
        file.close()  

In [0]:
def balance_train_and_test(source_train, source_test):
    
    for family in os.listdir(source_train):
        
        for classes in os.listdir(source_train+'/'+family):
            
            subfolder_train = source_train + '/' + family + '/' + classes
            subfolder_test = source_test + '/' + family + '/' + classes
            
            if not os.path.exists(subfolder_test):
                print(classes)
                os.mkdir(subfolder_test)
                pics_to_move = int(0.2*len(os.listdir(source_train + '/' + family + '/' + classes)))
                images = os.listdir(subfolder_train)
                
                for i in range(pics_to_move):
                    image = subfolder_train + '/' + images[i]
                    shutil.move(image, subfolder_test)

In [0]:
balance_train_and_test('sc5-tensorflow','sc5-test-tensorflow')
split_into_subfolders('sc5-Test/ground_truth.txt')
split_into_family_class('sc5/ground-truth-family.txt')
split_test_into_family_class('sc5-Test/ground-truth-family.txt')